In [1]:
import rqalpha as rqa
import rqalpha_plus
import rqoptimizer as rqo
import rqdatac as rqd
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import pickle
import os

In [2]:
rqd.init()

In [3]:
def get_best_size_fund(date,lower_limit,upper_limit):
    """获取某天基金规模在一定范围内的混合型和股票型基金代码"""
    fund_df = rqd.fund.all_instruments(date=date)
    fund_df = fund_df[fund_df['fund_type'].isin(['Hybrid','Stock'])].dropna()
    # 取规模在20-100亿之间的基金
    fund_df = fund_df[(fund_df['latest_size']>lower_limit) & (fund_df['latest_size']<upper_limit)]
    return fund_df['order_book_id'].tolist()

def get_best_profit_fund(fund_list,start_date,end_date,ratio=0.05):
    """获取基金业绩前5%的基金"""
    net_value_df = rqd.fund.get_nav(fund_list, start_date=start_date, end_date=end_date,expect_df=True)
    return_df = net_value_df.groupby(level=0).apply(lambda x:x.iloc[-1]/x.iloc[0]-1)
    n = int(return_df.shape[0]*ratio)
    return return_df.nlargest(n,'unit_net_value').index.tolist()

def get_target_portfolio(best_fund,date,nlargest,method,benchmark):
    """获得业绩最好的基金重仓股"""
    holdings_df = rqd.fund.get_holdings(best_fund, date=date)
    holdings_df = holdings_df[holdings_df['type']=='Stock']
    holdings_df = holdings_df.droplevel(1)[['order_book_id','weight']]
    # 获得每个基金前权重股
    max_weight_holdings = holdings_df.groupby(level=0).apply(lambda x:x.nlargest(nlargest,'weight'))
    if method == 'equally':
        index = np.unique(max_weight_holdings['order_book_id'].values)
        return pd.Series(1/len(index),index=index)
    elif method == 'mean':
        # 对每个股票取持有它的基金权重均值
        holdings_mean_weight = max_weight_holdings.groupby('order_book_id')['weight'].mean()
        # 权重归一
        return holdings_mean_weight/holdings_mean_weight.sum() 
    elif method == 'max':
        # 对每个股票取持有它的基金权重最大值
        holdings_mean_weight = max_weight_holdings.groupby('order_book_id')['weight'].max()
        # 权重归一
        return holdings_mean_weight/holdings_mean_weight.sum() 
    elif method == 'neutral':
        universe = np.unique(max_weight_holdings['order_book_id'].values)
        # 今天指数成分股
        components = rqd.index_components(benchmark,date)
        # 股票所属行业series
        industry = rqd.get_instrument_industry(components, source='sws', date=date)['first_industry_name']
        # 指数行业权重
        industry_weight = rqd.index_weights(benchmark,date).groupby(industry).sum()
        # 目标股票市值
        market_cap = rqd.get_factor(universe,'market_cap',date,date)
        # 股票行业分组组内市值加权
        universe_industry = rqd.get_instrument_industry(universe, source='sws', date=date)['first_industry_name']
        weight_in_industry = market_cap.groupby(universe_industry).apply(lambda x:x/x.sum())
        # 股票行业目标权重
        target_ids_weight = universe_industry.map(industry_weight)
        # 对于某些股票所属行业在中证500里面没有权重的设为0
        target_ids_weight.fillna(0, inplace=True)
        # 股票最终权重
        return (weight_in_industry*target_ids_weight).dropna()
    elif method == 'tracking_error':
        universe = np.unique(max_weight_holdings['order_book_id'].values)
        target_portfolio = rqo.portfolio_optimize(
            universe, date, rqo.MinTrackingError(),benchmark=benchmark
        )
        return target_portfolio.loc[lambda x: x != 0]

In [100]:
def init(context):
    all_dates = pd.date_range('20140331','20200903')
    trading_dates = rqd.get_trading_dates('20140331','20200903')

    rebalance_dates = []
    for i,date in enumerate(all_dates):
        date = date.date()
        if date.month in [4,7,10,1] and date.day==15:
            for j in range(5):
                date_ = date+timedelta(j)
                if date_ in trading_dates:
                    rebalance_dates.append(date_)
                    break

def handle_bar(context, bar_dict):
    # 如果是False(不是月初),直接返回
    if not _should_rebalance(context):
        return
    print("现有持仓数:", len(context.portfolio.positions))
    start_date = context.now-timedelta(365)
    end_date = context.now
    best_size = get_best_size_fund(context.now,context.lower_limit,context.upper_limit)
    best_profit = get_best_profit_fund(best_size,start_date,end_date,context.ratio)
    context.target_portfolio = get_target_portfolio(best_profit,context.now,**context.portfolio_args)
    rebalance(context, bar_dict)


def _should_rebalance(context):
    """判断今天是否调仓"""
#     # 获得下一个交易日
#     next_trading_day = rqd.get_next_trading_date(context.now)
#     # 判断是否是月初, 是月初返回True
#     is_rebalance = (next_trading_day.month != context.now.month) and (context.now.month in [3,6,9,12])
    return context.now.date() in rebalance_dates


def rebalance(context, bar_dict):
    positions = context.portfolio.positions
    for order_book_id in positions:
        if order_book_id not in context.target_portfolio:
            rqa.api.order_to(order_book_id, 0)
    
    # 对每个股票计算目标价值和当前价值的差值
    # 差值为正的是买单, 反之为卖单
    capital = context.portfolio.total_value * (1 - context.cash_cushion)
    to_sell, to_buy = {}, {}
    _money_for_one_lot = lambda order_book_id: bar_dict[order_book_id].close * 100
    for order_book_id, weight in context.target_portfolio.items():
        # 股票目标价值
        target_value = capital * weight
        # 目标和现有之差
        gap = target_value - positions[order_book_id].market_value
        # 买卖至少大于1手股票价值
        if abs(gap)<_money_for_one_lot(order_book_id):
            continue
        elif gap > 0:
            to_buy[order_book_id] = gap
        else:
            to_sell[order_book_id] = gap

    # to avoid liquidity issue, sell first, buy second
    for order_book_id, value in to_sell.items():
        rqa.api.order_value(order_book_id, value)
    for order_book_id, value in to_buy.items():
        rqa.api.order_value(order_book_id, value)

In [101]:
MILLION = 1_000_000
BILLION = 1000 * MILLION
benchmark = '000905.XSHG'
config = {
    "base": {
        "start_date": '2014-04-15',
        "end_date": '2020-09-03',
        "frequency": '1d',
        "accounts": {"stock": 0.1 * BILLION},
        "data_bundle_path":r'C:\Users\Administrator\.rqalpha-plus\bundle'
    },
    "mod": {
        "sys_analyser": {
            "enabled": True,
            "plot": True,
            "benchmark": benchmark,
        },
#         'sys_simulation': {
#             # 撮合时无视涨跌停
#             'price_limit': False,
#         },
#         'sys_risk': {
#             'enabled': True,
#             # 发单时无视停牌
#             'validate_is_trading': False,
#             # 发单时无视涨跌停
#             'validate_price': False
#         },
    },
    "extra": {
        "log_level": 'error',
        "context_vars": {
            'cash_cushion': 0.005,
            'lower_limit': 2e9,
            'upper_limit':1e10,
            'ratio':0.1,
            'portfolio_args':{
                'nlargest':10,
                'method':'tracking_error',
                'benchmark':benchmark
            }
        },
    },
}

In [ ]:
result = rqalpha_plus.run_func(init=init,handle_bar=handle_bar, config=config)

F:\anaconda\envs\ricequant\lib\site-packages\rqdatac\client.py:190: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)
[2014-04-15 15:00:00.000000] INFO: user_log: 现有持仓数: 0


WARN: aa_init returned NULL, no acceleration applied.


[2014-07-15 15:00:00.000000] INFO: user_log: 现有持仓数: 88


WARN: aa_init returned NULL, no acceleration applied.


[2014-10-15 15:00:00.000000] INFO: user_log: 现有持仓数: 118


...

[2015-01-15 15:00:00.000000] INFO: user_log: 现有持仓数: 124


WARN: aa_init returned NULL, no acceleration applied.


[2015-04-15 15:00:00.000000] INFO: user_log: 现有持仓数: 64


WARN: aa_init returned NULL, no acceleration applied.


[2015-07-15 15:00:00.000000] INFO: user_log: 现有持仓数: 106
